### **Lab session (PBL Week #9):**

This week we are going to look at the different methods to compute feature importance, and highilight some of their downsides.




### **Dataset :**

The dataset is part of the large dataset held at the National Institutes of Diabetes-Digestive-Kidney Diseases (NIH). The target variable is specified as "Outcome"; 1 indicates positive diabetes test result, 0 indicates negative.

### **Variables :**
* Pregnancies    : Number of pregnancies
* Glucose        : 2-hour plasma glucose concentration in the oral glucose tolerance test
* Blood Pressure : Blood Pressure (Smallness) (mm Hg)
* SkinThickness  : Skin Thickness
* Insulin        : 2-hour serum insulin (mu U/ml)
* Diabetes Pedigree Function : Function (2 hour plasma glucose concentration in oral glucose tolerance test)
* BMI            : Body mass index
* Age            : Age (years)
* Outcome        : Have the disease(1) or not (0)


Read more: https://www.sciencedirect.com/science/article/pii/S2352914819300176

### 🔎 1. Import Libraries, preprocessing, etc (Nothing to do)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# !pip install missingno
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

# some adjustments
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

from google.colab import files
uploaded = files.upload()


df_ = pd.read_csv('diabetes.csv')
df = df_.copy()

# auxiliary functions
def check_df(dataframe):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(3))
    print("##################### Tail #####################")
    print(dataframe.tail(3))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)


def grab_col_names(dataframe, cat_th=10, car_th=20):
    """


    ------
        dataframe: dataframe

        cat_th: int, optional

        car_th: int, optinal


    Returns
    ------
        cat_cols: list
                Categorical features
        num_cols: list
                Numerical features
        cat_but_car: list
               Categorical view cardinal variable list

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

def missing_values_table(dataframe, na_name=False):
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")
    if na_name:
        return na_columns

def missing_vs_target(dataframe, target, na_columns):
    temp_df = dataframe.copy()

    for col in na_columns:
        temp_df[col + '_NA_FLAG'] = np.where(temp_df[col].isnull(), 1, 0)

    na_flags = temp_df.loc[:, temp_df.columns.str.contains("_NA_")].columns

    for col in na_flags:
        print(pd.DataFrame({"TARGET_MEAN": temp_df.groupby(col)[target].mean(),
                            "Count": temp_df.groupby(col)[target].count()}), end="\n\n\n")

df.columns = [col.upper() for col in df.columns]
cat_cols, num_cols, cat_but_car = grab_col_names(df)


### 🔎 2. Missing Value Inputation (Nothing to do)

At a first glance there are no missing values. However, if you look at BLOODPRESSURE (or BMI or INSULIN for example) you can notice some values which do not make any sense.  



In [ ]:
df.isnull().sum()
#df=df.drop(["STDS: TIME SINCE LAST DIAGNOSIS", "STDS: TIME SINCE FIRST DIAGNOSIS", "HINSELMANN", "SCHILLER", "CITOLOGY"], axis=1)
df[["GLUCOSE","BLOODPRESSURE","SKINTHICKNESS","INSULIN","BMI"]]= df[["GLUCOSE","BLOODPRESSURE","SKINTHICKNESS","INSULIN","BMI"]].replace(0,np.NaN)
na_cols = missing_values_table(df, True)

#df.rename(columns={"BIOPSY": "OUTCOME"}, inplace=True)
#df = df.reindex(sorted(df.columns), axis=1)
df.head()



Replace missing data with median values (Nothing to do)

In [ ]:
def median_target(variable):
    temp = df[df[variable].notnull()]
    temp = temp[[variable, 'OUTCOME']].groupby(['OUTCOME'])[[variable]].median().reset_index()
    return temp

In [ ]:
columns = df.columns
columns = columns.drop("OUTCOME")

for col in columns:
    df.loc[(df['OUTCOME'] == 0) & (df[col].isnull()), col] = median_target(col)[col][0]
    df.loc[(df['OUTCOME'] == 1) & (df[col].isnull()), col] = median_target(col)[col][1]

In [ ]:
df = pd.get_dummies(df[cat_cols + num_cols], drop_first=True)

## 🔎 3. Feature importance

### 🦏 3.1 Logistic Regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression


In [ ]:
from sklearn.linear_model import LogisticRegression

y = df["OUTCOME"]
X=df.drop(["OUTCOME"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=17)

#Model 1
lr_model=LogisticRegression().fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
print("accuracy: ", accuracy_score(y_pred, y_test))

from sklearn.metrics import f1_score
print("f1-score:", f1_score(y_test,y_pred))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = lr_model.predict_proba(X_test)
y_pred_proba = probs[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
axes[0].plot(fpr,tpr,label="Logistic Regression, auc="+str(auc)[:5])
axes[0].legend(loc=4)


from sklearn.metrics import confusion_matrix
#Get the confusion matrix
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix, annot=True, ax=axes[1])
fig.tight_layout()
plt.show()


❗Plot the coefficients of the logistic regression model. Use the function bellow or create your own. (*See* https://pandas.pydata.org/docs/reference/api/pandas.Series.plot.barh.html for an alternative.)


❗Which is the most predictive feature according to the linear model?  

In [ ]:
def plot_importance(model, feature_names, num=len(X)):
    feature_imp = pd.DataFrame({'Value': model.coef_[0], 'Feature': feature_names})
    plt.figure(figsize=(8, 8))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                      ascending=False)[0:num])
    plt.title('Features Coeff')
    plt.tight_layout()
    plt.show()
    fig.tight_layout()

##Write your code here
## .......



❗ What happens if we standardize the features for the Linear model?

In [ ]:
rs = StandardScaler()
X_std=X.copy()
X_std[num_cols] = rs.fit_transform(X_std[num_cols])
X_std.head()

X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.30, random_state=17)

#Linear Model
lr_model=LogisticRegression().fit(X_train, y_train)
y_pred = lr_model.predict(X_test)
print("accuracy: ", accuracy_score(y_pred, y_test))

from sklearn.metrics import f1_score
print("f1-score:", f1_score(y_test,y_pred))

In [ ]:
#Plot the coefficients
## Write your code here. Use the attributes of the lr_model like in the previous cell
#linear_importances = pd.Series(????, index=X_train.columns).sort_values(ascending=True)
fig, ax = plt.subplots()
linear_importances.plot.barh(ax=ax, color='purple')
ax.set_title("Mean decrease in impurity")
fig.tight_layout()
plt.show()

### 🌴 3.2 Decision Trees

Train and evaluate a Random Forrest model here (Nothing to do)

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier().fit(X_train, y_train)


y_pred = rf_model.predict(X_test)
accuracy_score(y_pred, y_test)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = rf_model.predict_proba(X_test)
y_pred_proba = probs[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
axes[0].plot(fpr,tpr,label="Random Forrest, auc="+str(auc)[:5])
axes[0].legend(loc=4)
cf_matrix = confusion_matrix(y_test, y_pred)
sns.heatmap(cf_matrix, annot=True, ax=axes[1])
fig.tight_layout()
plt.show()


Mean Decrease in Impurity (MDI or Gini Importance) approximates each feature importance as the sum over the number of splits (across all tress) that include the feature, proportionally to the number of samples it splits.

❗Measure the MDI of the trained RF model.

*Hint:* Use directly **feature_importances_** attribute from the RF model: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html





In [ ]:
from sklearn import tree

#First let's plot a tree from the RF

#fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
_ = tree.plot_tree(rf_model.estimators_[5],
                   feature_names=X_train.columns,
                   class_names=["Diabetes", 'Normal'],
                   filled=True)


# Put the manes of the features in a list
feature_names = X_train.columns

## Complete code here. Get the importances from the rf_model
## importances = ....

#Create a pandas series with the MDI importances and sort them
forest_importances = pd.Series(importances, index=feature_names).sort_values(ascending=True)

#Plot the importances
fig, ax = plt.subplots()
forest_importances.plot.barh(ax=ax, color='purple')
ax.set_title("Mean decrease in impurity")
fig.tight_layout()
plt.show()

🎲 **Random features**  

The impurity-based feature importance of random forests suffers from being computed on statistics derived from the training dataset. MDI feature importance are equally susceptible to high cardinal features. They can aslo inflate the importance of numerical features.
https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html

❗ Add a feature with random real values (high cardinality)  

❗ Add a feature with random integer values (3 values, low cardinality)

*Hint:* use the available methods in numpy:
https://numpy.org/doc/1.14/reference/generated/numpy.random.RandomState.html

❗ Retrain and evaluate the model.


In [ ]:
# Create a copy of the X dataset
X_modified = X.copy()

#Add random number features using np.random
rng = np.random.RandomState()

## Write your code here. Use RNG to generate a random feature (similar to below))
# X_modified[... = ...

##  Use RNG to generate a random integer feature
X_modified["RANDOM_CAT"] = rng.randint(3, size=X.shape[0])


# Split the set again
X_train_rand, X_test_rand, y_train_rand, y_test_rand = train_test_split(X_modified, y, test_size=0.30, random_state=23)
#Fit a classifier
rf_model_rand = RandomForestClassifier().fit(X_train_rand, y_train_rand)


y_pred_rand = rf_model_rand.predict(X_test_rand)
accuracy_score(y_pred_rand, y_test)
#Plot the accuracy
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = rf_model_rand.predict_proba(X_test_rand)
y_pred_proba = probs[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test_rand,  y_pred_proba)
auc = metrics.roc_auc_score(y_test_rand, y_pred_proba)
axes[0].plot(fpr,tpr,label="Random Forrest, auc="+str(auc)[:5])
axes[0].legend(loc=4)
cf_matrix = confusion_matrix(y_test_rand, y_pred_rand)
sns.heatmap(cf_matrix, annot=True, ax=axes[1])
fig.tight_layout()
plt.show()

❗ Plot the MDI feature importances. What do you notice?

In [ ]:
feature_names = X_train_rand.columns
mdi_importances = pd.Series(
    rf_model_rand.feature_importances_, index=feature_names
).sort_values(ascending=True)


fig, ax = plt.subplots()
## Write your code here ...
### .....
ax.set_title("Feature importances using MDI")
fig.tight_layout()
plt.show()

### 🪓 3.3 Drop features

**❗Compute feature importance by leaving out the features one by one**

Plot the distribution of some of the attributes side by side of the 2 training datasets.

**❗Which feature seems to be the most important?**

In [ ]:


# Split the set again and fit the model (to remove the previous random features)
#X=X.drop(["RANDOM_CAT", "RANDOM_NUM"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=14)
rf_model = RandomForestClassifier().fit(X_train, y_train)


❗ Complete the function that implements the feature importance based on dropping features from the dataset.

In [ ]:
def drop_features(features,model=RandomForestClassifier(random_state=78), X_train=X_train, y_train=y_train,
                  X_test=X_test, y_test=y_test):

    # Train the model with all features first
    initial_model = model.fit(X_train, y_train)
    y_pred = initial_model.predict(X_test)
    # compute its accuracy on the test set
    initial_accuracy = accuracy_score(y_pred, y_test)
    drop_importances = [] # a list which will contain the drop feature importances
    # Remove each feature from the dataset one by one
    for feature in features:
      # Remove each feature from the train dataset and test dataset one by one
      # !!! Write your code here:
      ## X_train_drop = ??
      ## X_test_drop = ??
      # train a new model without the feature dropped
      rf_model = model.fit(X_train_drop, y_train)

      #compute the model's accuracy on the test set
      y_pred_drop = rf_model.predict(X_test_drop)
      drop_accuracy = accuracy_score(y_pred_drop, y_test)
      #compute th difference in the accuracy with respect to the initial model (initial_accuracy)
      # !!! Write your code here ...
      # .. drop_importance = ???
      # add the difference to the list
      drop_importances.append(drop_importance)

    return pd.Series(drop_importances, index=features).sort_values(ascending=True)

drop_imp = drop_features(X_train.columns)
ax =drop_imp.plot.barh(color='green')
ax.set_title("Drop columns importance")
fig.tight_layout()
plt.show()

### 🎲 3.4 Permutation importance

Permutation feature importance measures the decrease in the model accuracy of the model after we permute the feature’s values, breaking the relationship between the feature and the outcome.

*Read more:*

https://christophm.github.io/interpretable-ml-book/feature-importance.html


❗ Compute the permutation feature importance on the test set. *Hint*: use the available implementation ind sklearn:

https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html

❗Which feature is the most predictive according to this metric?

In [ ]:
from sklearn.inspection import permutation_importance
#rf_model = RandomForestClassifier().fit(X_train, y_train)
feature_names = X_train.columns

## Compute the PI on the test set using the sklearn method. !!!! Write your code here
## result = ....


importances = pd.DataFrame(
    result.importances.T,
    columns=feature_names,
)

ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()
plt.show()

### 👯 3.5 Correlated features

Highly correlated features can decrease the  importance of these features, when dropping features one by one. Here, we simulate artificial correlated features, by simply duplicating one of the important features (columns) in the dataset.

❗ Use the drop feature approach on this new dataset. How does the importance change as compared to the initial set-up?

In [ ]:
X_duplicate=X.copy()


## Write your code here
## Add a 'INSULIN_copy' feature to the X_duplicate dataset
## ...

X_train, X_test, y_train, y_test = train_test_split(X_duplicate, y, test_size=0.30, random_state=41)
print(X_train.columns)


## Write your code here...Call the drop_features function as before
## drop_imp = ....

ax =drop_imp.plot.barh(color='darkorange')
ax.set_title("Drop columns importance")
fig.tight_layout()
plt.show()


Correlated features can decrease the permutation importance of these features by splitting the importance between both features. Equally,  permutation feature importance can be biased by unrealistic data instances. *Read more:*

https://christophm.github.io/interpretable-ml-book/feature-importance.html#disadvantages-9

❗Compute the permutation importance for the new dataset that contains an identical copy of one the improtant features. How does the importance change with respect to the initial set up?

In [ ]:

## We fit a RF classifier on this new dataset
rf_model_perm = RandomForestClassifier().fit(X_train, y_train)
### Write your code here ... (similar to 3.4)


❗[OPTIONAL] Try out other models, SVM, MLP, etc...

## 🍋 4. LIME:

Local interpretable model-agnostic explanations (LIME) focuses on training local surrogate models to explain individual predictions.

*Read more:*

https://christophm.github.io/interpretable-ml-book/lime.html



In [ ]:

!pip install lime
import lime
import lime.lime_tabular

explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.values.tolist(),
                                                  verbose=True, mode='classification')

❗ Chose an instance from the test set (X_test) and explain the random forest's prediction.

Check out tutorials on using LIME:
https://github.com/marcotcr/lime


Check out the explain_instance method from the lime documentation: https://lime-ml.readthedocs.io/en/latest/lime.html





In [ ]:
exp = explainer.explain_instance(X_test.values[90], rf_model.predict_proba)
exp.show_in_notebook(show_table=True)

## 5. Interpretability with SHAP
SHaply Additive exPlanations (SHAP) is a model-agnostic interpretability framework developed to understand the contributions and effects of features on the model outputs. SHAP values help explain how features contribute to model outputs at the local (prediction) level. SHAP does not globally interpret how the model makes its decisions, but rather provides a post-hoc explanation of what features were used and how for different predictions.

More on SHAP can be found in the ML interpretability book:
https://christophm.github.io/interpretable-ml-book/local-methods.html

### 5.0 Define and train a model (if don't have one already)
Example with an XGBoost model.

In [ ]:
# train an XGBoost model
from xgboost import XGBClassifier

xgb = XGBClassifier(objective='binary:logistic')
xgb.fit(X_train, y_train)

### 5.1 Compute and inspect SHAP values

In [ ]:
# install shap libraries
!pip install shap
import shap
shap.initjs() # needed for visualisations

Create SHAP explainer, try it with different models (logistic regression, decision tree, xg boost...).

> Note: in a random forest, binary classification is sometimes treated as multi-class classification, in which case it is easier to pick a single class to do the SHAP value analysis on. Since this is binary classification, the positive/negative class results are complementary. In the example below, we pick the positive class for the SHAP value analysis.


In [ ]:
## for a linear model
# explainer = shap.LinearExplainer(lr_model, X_train)
# shap_values = explainer.shap_values(X_test)

## for a tree-based model (decision tree, xgboost, random forest)
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)
print(shap_values.shape)

# pick positive class for SHAP analysis
shap_values_positive_class = shap_values[:, :, 1]
print(shap_values_positive_class.shape)

Expected value of the model (if multi-class classification, you will get the expected value for each class).

In [ ]:
print(explainer.expected_value)

### 5.2 Feature importance using SHAP values
SHAP values can first be used to understand feature importance by simply looking at the magnitude of the SHAP value. A higher SHAP value indicates a heavier contribution to the model's decision, and therefore more importance.

First, we look at a simple barplot of the features, where importance is dictated by the magnitude of the SHAP value.

In [ ]:
shap.summary_plot(shap_values_positive_class, X_test, plot_type="bar")

For a more detailed analysis, we can look at density SHAP summary plot (same plot but bars are turned into actual values).

Each point in the SHAP summary plot represents a **single prediction** made by the model. The colour of each point corresponds to the value of the feature for that prediction. The position on the x-axis shows the SHAP value, which indicates how much each feature contributes to the model's output for that prediction.

We can use a SHAP summary plot to identify which features that have a strong influence (large absolute SHAP) and which ones are less impactful (SHAP near 0).
The colour of the points helps us understand how the feature's value (high/low) is related to the feature's influence on the model's decision.

> Which features have the most significant impact on the model's predictions based on the SHAP summary plot? Is that consistent to what you observed before?


> What does the distribution of SHAP values tell you about the variability of each feature's impact on the model output? Do features have a consistent impact across different predictions?

In [ ]:
shap.summary_plot(shap_values_positive_class, X_test)

### 5.3 Taking a closer look at predictions

#### 5.3.1 Individual predictions using force plots

SHAP force plots help explain the **contribution of individual features** to a **specific prediction**.

The base value (expected value) is the starting point of the model's prediction.

Features either push the prediction higher (positive SHAP values) or lower (negative SHAP values).
The final prediction is the sum of the base value and all SHAP contributions.

**Colours** indicate the direction:
- Red: feature pushes the prediction higher.
- Blue: feature pushes the prediction lower.

The length of each feature **arrow** reflects how much it contributes to the decision (how much is pushed away from base value).

In [ ]:
# look at some individual predictions
from IPython.display import display # need to explicitly display force plot in a for loop

for i in range(5,10):
  y_predicted = y_test.iloc[i]
  print('Prediction: ', y_predicted)
  shap.initjs()
  force_plot = shap.force_plot(explainer.expected_value[1],
  shap_values_positive_class[i,:], X_test.iloc[i,:])
  display(force_plot)



#### 5.3.2 Visualise multiple predictions by stacking force plots

If we pass in all `shap_values` and `X_test`, we get a nice interactive plot that includes all the test points.

> Note: the samples are ordered by similarity in their features, but you can change the sample ordering in the drop-down menu (both in x and in y)
Example: if we pick "sample order by output value", we see a nice differentiation between outputs and the role of a key feature - which one?

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values_positive_class[:len(X_test),:], X_test.iloc[:len(X_test),:])